In [ ]:
from bs4 import BeautifulSoup
import requests
import gspread
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from gspread_dataframe import set_with_dataframe
import requests
import selenium.webdriver as webdriver
import time
import numpy as np 
import pandas as pd

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [ ]:
City=["Mumbai","Bangalore","Delhi"]
speciality=["Dentist","Gynecologist","Pediatrician","Orthopedist","Dermatologist","Physiotherapist","Infertility Specialist","Ophthalmologist","Urologist","Psychiatrist","Cardiologist","Dietitian","Gastroenterologist","Neurologist","Neurosurgeon","Pulmonologist","Bariatric Surgeon","Rheumatologist","Chiropractor"]
pg_link="https://www.practo.com/{City}/doctors?page={page}"

In [ ]:
pg=range(1,1001)
docs=set()
cnt=0
for cty in City:
#     for spc in speciality:
    for c in pg:
#             print(cty,spc,"Page",c)
        try:
            r=requests.get(pg_link.format(City=cty,page=c),headers=headers)
            htmlcontent=r.content
            soup=BeautifulSoup(htmlcontent,"html.parser")

    #         divs=soup.find_all('div', attrs={'class' :'info-section'})
            for data in soup.find_all('div', class_='info-section'):
                for a in data.find_all('a'):
                    if "practice_id" in a.get('href') and "recommended" not in a.get('href') :
                        docs.add(("https://www.practo.com/"+a.get('href')))
        except:
            print("in",cty,"for",c,"page not found",end=" ")
            pass
#             if c==3:
#                 break
    print(cty,": ","Extracted",abs(cnt-len(docs)))
    cnt=len(docs)
#         break
    
                    
    

    


Mumbai :  Extracted 1944
Bangalore :  Extracted 3176
Delhi :  Extracted 2207


In [ ]:
len(docs)

7327

In [ ]:
for i in sorted(list(docs)):
    print(i)

In [ ]:
print(len(docs))

7327


In [ ]:
doc_details={"name":[],"speciality_of_doctor":[],"degree_type":[],"year_of_experiance":[],"Location":[],"city":[],"dp_score":[],"npv":[],"consult_fee":[]}
a=0
for doc in docs:
    a=a+1
    r=requests.get(doc,headers=headers)
    htmlcontent=r.content
    soup=BeautifulSoup(htmlcontent,"html.parser")
    lst=[]
    try:
        doc_details["name"].append(soup.find("h1",class_="c-profile__title").text)  #Name
    except:
        doc_details["name"].append(np.NaN)
    try:
        doc_details["Location"].append(soup.find("h4").text)
    except:
        doc_details["Location"].append(np.NaN)
        
    try:    
        for i in soup.find_all("h2"):
            lst.append(i.text)
        doc_details["city"].append(lst[1])
        doc_details["speciality_of_doctor"].append(lst[2])
    except:
        doc_details["city"].append(np.NaN)
        doc_details["speciality_of_doctor"].append(np.NaN)
        
    try:
        doc_details["dp_score"].append(soup.find("span",class_="u-green-text").text)
    except:
        doc_details["dp_score"].append(np.NaN)
    lk=0
    for k in lst:
        lk=lk+1
        try:
            if  "Experience Overall" in k:
                doc_details["year_of_experiance"].append(k)
#                 print(lk, len(lst))
                break
        except:
            pass
    if lk==len(lst):
        print(a)
        doc_details["year_of_experiance"].append(np.NaN)
        
    try:
        doc_details["degree_type"].append(soup.find_all("p")[0].text)
    except:
        doc_details["degree_type"].append(np.NaN)
    
    try:
        doc_details["npv"].append(soup.find("span",class_="u-smallest-font").text)
    except:
        doc_details["npv"].append(np.NaN)
    try:
        doc_details["consult_fee"].append(soup.find_all("span",class_="u-f-right")[-1].text)
    except:
        doc_details["consult_fee"].append(np.NaN)
    
    
        
#     if a==10:
#         break
#     for i in soup.find_all("p"):
#         print(i.text)
# print(lst)

1233
3558
4544
4567
5002


In [ ]:
print(lst)

['Home', 'Mumbai', 'Dentist', 'Orthodontist', 'Ghatkopar East', 'Dr. Sheel Vora', 'Orthodontist', 'Cosmetic/Aesthetic Dentist', '17\xa0Years Experience Overall\xa0 (10 years as specialist)', "Dr. Vora's Dental Care Center", 'Patient Stories for Dr. Sheel Vora', 'Common questions & answers', 'Services', 'Specializations', 'Awards and Recognitions', 'Education', 'Memberships', 'Experience', 'Registrations', 'Pick a time slot', 'Top doctor specialities in Mumbai']


In [ ]:
for i in doc_details:
    print(len(doc_details[i]))
    

7327
7327
7327
7327
7327
7327
7327
7327
7327


In [ ]:
pd.DataFrame(doc_details)

,name,speciality_of_doctor,degree_type,year_of_experiance,Location,city,dp_score,npv,consult_fee
0,Dr. Harish K C,Gastroenterologist,"MBBS, MD - General Medicine, DM - Gastroentero...",12 Years Experience Overall (4 years as speci...,"Ashoknagar, Bangalore",Bangalore,93%,(27 votes),₹ 800 fee
1,Dr. Deepa Raje Nimbalkar,Homoeopath,"BHMS, MD - Homeopathy, MD - Pediatrics",18 Years Experience Overall,"HSR Layout, Bangalore",Bangalore,96%,(117 votes),₹ 600 fee
2,Dr. Rupal V Rana,General Physician,MBBS,21 Years Experience Overall,"Borivali West, Mumbai",Mumbai,97%,(34 votes),₹ 2130 fee
3,Dr. Khushboo Nikhil Kedia,Dentist,"BDS, Comprehensive Course in Modern Endodontics",14 Years Experience Overall,"Mulund West, Mumbai",Mumbai,92%,(13 votes),₹ 300 fee
4,Dr. Shashi Kumar B,Dentist,BDS,7 Years Experience Overall,"Hebbal, Bangalore",Bangalore,100%,(22 votes),₹ 200 fee
...,...,...,...,...,...,...,...,...,...
7322,Dr. Shreedevi Tanksale,Gynecologist/obstetrician,"DGO, DNB - Obstetrics & Gynecology, MBBS",14 Years Experience Overall (8 years as speci...,"Vileparle East, Mumbai",Mumbai,99%,(164 votes),₹ 1000 fee
7323,Dr. Kumara Swamy,Oncologist,"MBBS, Diploma in Radio Therapy, DNB - Radiothe...",45 Years Experience Overall (37 years as spec...,"Hebbal, Bangalore",Bangalore,NaN,In-clinic,₹ 900 fee
7324,Dr. Kamal Bachani,Orthopedist,"MBBS, MS - Orthopaedics",32 Years Experience Overall (28 years as spec...,"Malviya Nagar, Delhi",Delhi,96%,(426 votes),₹ 1500 fee
7325,Dr. Harish N S,General Surgeon,"MBBS, MS - General Surgery",15 Years Experience Overall (9 years as speci...,"Basavanagudi, Bangalore",Bangalore,98%,(103 votes),₹ 700 fee


In [ ]:
pd.DataFrame(doc_details).to_csv("practo_dataset.csv")